
The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___


___

Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.


In [131]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [134]:
import pandas as pd
import numpy as np

def blight_model():
    from sklearn.ensemble import GradientBoostingClassifier
    
    #Read train and test data
    train_data = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test_data = pd.read_csv('test.csv')
    
    #Filter the violators that were found not responsibles
    train_data = train_data[(train_data.loc[:, "compliance"] == 1.0) | (train_data.loc[:, "compliance"] == 0.0)]
    
    #Delete columns that are not included in test set
    columns = list(set(train_data.columns).difference(test_data.columns))
    train_data = train_data.drop(['payment_status', 'payment_date', 'balance_due', 'collection_status', 'payment_amount', 'compliance_detail'], axis=1)
    
    #Delete string columns
    columns_to_delete = ['violator_name', 'zip_code', 'country', 'city',
        'inspector_name', 'violation_street_number', 'violation_street_name',
        'violation_zip_code', 'violation_description',
        'mailing_address_str_number', 'mailing_address_str_name',
        'non_us_str_code', 'agency_name', 'state', 'disposition',
        'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code']
    train_data = train_data.drop(columns_to_delete, axis=1)
    
    columns_to_delete_test = ["agency_name", "violator_name", "inspector_name", "violation_street_name", 'violation_zip_code', 'violation_description',
        'mailing_address_str_number', 'mailing_address_str_name', "city", 'state', "zip_code", 'disposition',"non_us_str_code","country",
        'ticket_issued_date', 'hearing_date', 'grafitti_status', 'violation_code', "violation_street_number"]
    test_data = test_data.drop(columns_to_delete_test, axis=1)
    
    #Read address and latlons data
    address =  pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')

    #join latlons and address data
    address = address.set_index('address').join(latlons.set_index('address'), how='left')

    # Join address and location to train and test data
    train_data = train_data.set_index('ticket_id').join(address.set_index('ticket_id'))
    test_data = test_data.set_index('ticket_id').join(address.set_index('ticket_id'))
    
    #Fill NaN values in lat and lon
    train_data.lat.fillna(method='pad', inplace=True)
    train_data.lon.fillna(method='pad', inplace=True)
    test_data.lat.fillna(method='pad', inplace=True)
    test_data.lon.fillna(method='pad', inplace=True)
    
    #Define X and Y
    X_train = train_data.drop("compliance", axis=1)
    Y_train = train_data["compliance"]
    X_test = test_data
    
    #Build and train the model
    clf = GradientBoostingClassifier()
    clf = clf.fit(X_train, Y_train)
    
    #Predict
    pred = clf.predict_proba(X_test)
    Y_test = pred[:,1]
    
    #Show predicted probabilities
    Y_index = X_test.index
    res = pd.Series(list(Y_test), index =list(Y_index)) 
      
    return(res)

In [135]:
blight_model()

284932    0.055029
285362    0.020772
285361    0.057549
285338    0.081691
285346    0.089565
285345    0.081691
285347    0.082342
285342    0.340760
285530    0.032698
284989    0.035218
285344    0.078192
285343    0.034596
285340    0.036524
285341    0.082342
285349    0.089565
285348    0.081691
284991    0.035218
285532    0.035153
285406    0.026362
285001    0.061449
285006    0.053407
285405    0.020772
285337    0.028773
285496    0.059519
285497    0.060923
285378    0.025739
285589    0.029293
285585    0.053553
285501    0.064882
285581    0.024267
            ...   
376367    0.027275
376366    0.032629
376362    0.035159
376363    0.059203
376365    0.027275
376364    0.032629
376228    0.059141
376265    0.029526
376286    0.312537
376320    0.036412
376314    0.027891
376327    0.386180
376385    0.333487
376435    0.461091
376370    0.368065
376434    0.064274
376459    0.069496
376478    0.018543
376473    0.035931
376484    0.038145
376482    0.032820
376480    0.